In [7]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import avg, col, desc


24/03/31 16:16:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [9]:
!echo $PYSPARK_PYTHON

In [10]:
!echo $PYSPARK_DRIVER_PYTHON

In [14]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [15]:
sys.executable

'/opt/anaconda3/envs/dtezoomcamp/bin/python'

In [16]:
!echo $PYSPARK_DRIVER_PYTHON

/opt/anaconda3/envs/dtezoomcamp/bin/python


In [17]:
!echo $PYSPARK_PYTHON

/opt/anaconda3/envs/dtezoomcamp/bin/python


In [18]:
# Start spark session
pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GeeksForGeeksTestGroupByWindow") \
    .config("spark.jars.packages", kafka_jar_package) \
    .getOrCreate()

24/03/31 16:20:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [35]:
from pyspark.sql.functions import row_number
from pyspark.sql import Window
df = spark.range(3)
w = Window.orderBy(df.id.desc())
df.withColumn("desc_order", row_number().over(w)).show()

24/03/31 16:33:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/31 16:33:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/31 16:33:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/31 16:33:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/31 16:33:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---+----------+
| id|desc_order|
+---+----------+
|  2|         1|
|  1|         2|
|  0|         3|
+---+----------+



In [52]:
 # Define sample data
simpleData = [("Pulkit","trial_1",32, '2024-03-31 18:37:51.068331'),
    ("Ritika","trial_1",42, '2024-03-31 18:37:52.068331'),
    ("Pulkit","trial_2",45, '2024-03-31 18:37:53.068331'),
    ("Ritika","trial_2",42, '2024-03-31 18:37:54.068331'),
    ("Ritika","trial_3",75, '2024-03-31 18:37:55.068331'),
    ("Pulkit","trial_3",55, '2024-03-31 18:37:56.068331'),
    ("Ritika","trial_4",75, '2024-03-31 18:37:57.068331'),
    ("Pulkit","trial_4",32, '2024-03-31 18:37:58.068331')
  ]
 
# define the schema
schema = ["Name","Number_of_Trials","Marks", "timestamp"]
 
# create a dataframe
df = spark.createDataFrame(data=simpleData, schema = schema)
# w = Window.orderBy(df.Name.desc())
# df = df.withColumn("row_num", row_number().over(w))
# df = df.withColumn('timestamp', (F.current_timestamp() + F.expr('INTERVAL 2 HOURS')).cast('string')) 
df.show(truncate=False)

+------+----------------+-----+--------------------------+
|Name  |Number_of_Trials|Marks|timestamp                 |
+------+----------------+-----+--------------------------+
|Pulkit|trial_1         |32   |2024-03-31 18:37:51.068331|
|Ritika|trial_1         |42   |2024-03-31 18:37:52.068331|
|Pulkit|trial_2         |45   |2024-03-31 18:37:53.068331|
|Ritika|trial_2         |42   |2024-03-31 18:37:54.068331|
|Ritika|trial_3         |75   |2024-03-31 18:37:55.068331|
|Pulkit|trial_3         |55   |2024-03-31 18:37:56.068331|
|Ritika|trial_4         |75   |2024-03-31 18:37:57.068331|
|Pulkit|trial_4         |32   |2024-03-31 18:37:58.068331|
+------+----------------+-----+--------------------------+



In [54]:
df \
  .groupBy(
    F.window(df.timestamp, "5 minutes"),
    col("Marks"))\
  .count()\
  .show(truncate=False)

+------------------------------------------+-----+-----+
|window                                    |Marks|count|
+------------------------------------------+-----+-----+
|{2024-03-31 18:35:00, 2024-03-31 18:40:00}|32   |2    |
|{2024-03-31 18:35:00, 2024-03-31 18:40:00}|42   |2    |
|{2024-03-31 18:35:00, 2024-03-31 18:40:00}|45   |1    |
|{2024-03-31 18:35:00, 2024-03-31 18:40:00}|75   |2    |
|{2024-03-31 18:35:00, 2024-03-31 18:40:00}|55   |1    |
+------------------------------------------+-----+-----+



In [55]:
df \
  .groupBy(
    F.window(df.timestamp, "5 seconds"),
    col("Marks"))\
  .count()\
  .show(truncate=False)

+------------------------------------------+-----+-----+
|window                                    |Marks|count|
+------------------------------------------+-----+-----+
|{2024-03-31 18:37:50, 2024-03-31 18:37:55}|32   |1    |
|{2024-03-31 18:37:50, 2024-03-31 18:37:55}|42   |2    |
|{2024-03-31 18:37:50, 2024-03-31 18:37:55}|45   |1    |
|{2024-03-31 18:37:55, 2024-03-31 18:38:00}|75   |2    |
|{2024-03-31 18:37:55, 2024-03-31 18:38:00}|55   |1    |
|{2024-03-31 18:37:55, 2024-03-31 18:38:00}|32   |1    |
+------------------------------------------+-----+-----+



In [56]:
spark.stop()